# Text Analytics Coursework -- Tasks 1.1 to 1.3

# Instructions:

This notebook is forms the first part of your coursework assignment for Text Analytics in Spring 2025. You will need to read the instructions below and complete numbered tasks indicated by "TASK n". To complete the tasks, you will write code or explanations between the comments "#WRITE YOUR ANSWER HERE" and "#END OF ANSWER". For example:

TASK 0: Complete the function below to output "hello world". 


In [1]:
def demo_fun():
    # WRITE YOUR ANSWER HERE
    print("hello world")
    # END OF ANSWER

### DO NOT MODIFY
demo_fun()
###

hello world


There is also some code in the cell that should not be modified. This code saves your results to file in the correct format, which is necessary for us to be able to mark your answers. Before you submit your notebook, please make sure this code has not been modified, then restart your kernel, clear all cell outputs, run all of your code once again, then save the notebook. 

Please note:
  * The notebook you upload must include all the saved cell output after running all cells.
  * The notebook code must be complete so that it reproduces all your output when we run it. 
  * For this coursework, we recommend that you use your virtual environment that you created for the labs. The packages you need are: numpy, scipy, nltk, pytorch, transformers and datasets (from HuggingFace), pandas, matplotlib and scikit-learn. 

## Marking guidelines:
1. This notebook is worth 32% of the marks for the Text Analytics assignment.
1. The number of marks for each task is shown alongside the task.
1. We will evaluate the output of your code after running it, and marks will be awarded based on how well the output matches the task's instructions. 
1. We will give partial marks for incomplete or partially correct answers. 
1. We do not give additional marks for code style or comments, but clear code will help us to understand what you have done so that we can award partial marks where necessary. 
1. Unless the task asks you to implement something from scratch, there is no penalty for using software libraries in your implementation.

## Support:

The main source of support will be during the lab sessions. The TAs and lecturers will help you with questions about the lectures, the code provided for you in this notebook, and general questions about the topics we cover. For the assessed tasks, they can only answer clarifying questions about what you have to do. Please email Edwin if you have any other queries edwin.simpson@bristol.ac.uk and/or post your query to the Teams channel for this unit.

## Deadline:

The notebook must be submitted along with the second notebook on Blackboard before **Monday 28th April at 13.00**. 

## Submission:

For this part of the assignment, please zip up the folder containing this file and the 'outputs' directory, containing the output from this notebook as .csv files. Please name your notebook file like this:
   * Name this notebook 'text_analytics_part1_\<student number\>.ipynb'. Replace '\<student number\>' with your student number, which consists only of digits beginning with '2'. 
   * We mark anonymously, so please don't include your name in the notebook.

You can submit the file on Blackboard to the submission point "Text Analytics Part 1 Notebook". Remember that the assignment also has parts 2 and 3, described in the PDF file on Blackboard.

# Setup: random seeds

Each student will work with slightly different data splits and model weights, which will be determined by setting your 'random seed'. 
We will check that your results come from using your random seed. Please set the seed in the cell below by changing the value of 'my_student_number' to your own student number (not your username, the number you can see on eVision that contains only digits). 

Using the correct seed ensures that your results are reproducible when we rerun your notebook.

In [2]:
import torch
import random
import numpy as np

def set_seed(seed: int = 42):
    random.seed(seed)  # Python's built-in random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (if available)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable benchmark mode for reproducibility

### SET YOUR SEED TO YOUR STUDENT NUMBER HERE
my_student_number = 2619208
set_seed(my_student_number)

# Setup: loading the data

Let's make a folder to save the output of your work:

In [3]:
import os
import pandas as pd

os.mkdir('./outputs')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './outputs'

Now, let's load some more packages we will need later:

In [6]:
%load_ext autoreload
%autoreload 2

# Use HuggingFace's datasets library to access the Emotion dataset
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split 


The dataset classifies paragraphs taken from corporate disclosures that discuss climate-related issues. It classifiers them into "risk" (0), "neutral" (1) or "opportunity" (2) representing the sentiment of the paragraph.

First we need to load the data. The data is already split into train, validation and test. The _validation_ set (also called 'development' set or 'devset') can be used to compute performance of your model when tuning hyperparameters, optimising combinations of features, or looking at the errors your model makes before improving it. This allows you to hold out the test set (i.e., not to look at it at all when developing your method) to give a fair evaluation of the model and how well it generalises to new examples. This avoids tuning the model to specific examples in the test set. An alternative approach to validation is to not use a single fixed validation set, but instead use [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html). 

In [7]:
cache_dir = "./data_cache"

# load the original training set from HuggingFace
train_dataset = load_dataset(
    "climatebert/climate_sentiment",
    split="train",
    cache_dir=cache_dir,
)

# we're going to create a new validation set by splitting the data
dataset_splits = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset_splits["train"]
val_dataset = dataset_splits["test"]

train_texts = np.array(train_dataset["text"])
val_texts = np.array(val_dataset["text"])

train_labels = np.array(train_dataset["label"])
val_labels = np.array(val_dataset["label"])

print(f"Training dataset with {len(train_texts)} instances loaded")
print(f"Development/validation dataset with {len(val_texts)} instances loaded")


### DO NOT MODIFY
# save gold labels to file
pd.DataFrame(val_labels).to_csv('./outputs/val_labels.csv')

Training dataset with 800 instances loaded
Development/validation dataset with 200 instances loaded


In this notebook, you're going to build three different classifiers for this dataset, then compare how they work, and analyse the results. We are going to start by implementing a naïve Bayes classifier from scratch. 

We are going to begin by initialising some useful variables and doing some very simple pre-processing using CountVectorizer.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

num_classes = 3

def preprocess(train_texts):
    vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=word_tokenize)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess(train_texts)

C:\Users\agarw\anaconda3\envs\text_analytics\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## TASK 1.1a

Complete the function below to compute the class priors, $p(y_n = c)$ for each class label $c$, where $y_n$ is the class label of document $n$. Do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the class priors as a list or Numpy array containing the probabilities. You do not need to apply any smoothing or regularisation.    (3 marks)

In [9]:
def compute_class_priors(texts, labels):
    priors = np.zeros(num_classes)

    ### WRITE YOUR ANSWER HERE
    total_docs = len(labels)  # Total number of documents
    for i in range(num_classes):
        priors[i] = np.sum(labels == i) / total_docs  # Compute probability of each class
    ### END OF ANSWER
    return priors


class_priors = compute_class_priors(train_texts, train_labels)
print(class_priors)

### DO NOT MODIFY
pd.DataFrame(class_priors).to_csv('./outputs/11a_class_priors.csv')

[0.33375 0.4075  0.25875]


## TASK 1.1b

Complete the function below to extract n-gram features from the text, then compute the liklihood $p(x_{ni} = w | y_n = c)$ that the $i$ th n-gram in document $n$ is $w$, given that the class of $n$ is $c$. Again, do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the likelihoods as a 2D Numpy array containing probabilities. You should apply smoothing by adding counts of +1 to the counts of each feature.  (3 marks)

In [10]:
def compute_feature_likelihoods(X, labels):

    likelihoods = np.ones((num_features, num_classes))  # a 2D numpy array where you can store the likelihoods. Note that all values are initialised to one.

    ### WRITE YOUR ANSWER HERE
    labels = np.array(labels)
    feature_counts = np.zeros((num_features, num_classes))  # start with 0s
    
    for c in range(num_classes):
        class_indices = X[labels == c]  # Get indices of samples in class c
        
        class_counts = np.sum(class_indices, axis=0)          
        feature_counts[:, c] = class_counts  # Sum feature occurrences in class
    
        # Normalize to compute probabilities
    likelihoods=(feature_counts + 1) / (np.sum(feature_counts, axis=0) + num_features)  # Apply smoothing in denominator

    ### END OF ANSWER
    return likelihoods

likelihoods = compute_feature_likelihoods(X, train_labels)

### DO NOT MODIFY
pd.DataFrame(likelihoods).to_csv('./outputs/11b_likelihoods.csv')

Now, we are going to use the code in the next cell to compute the log probabilities of each class for each text in the validation set. This code will use the previous functions you implemented, compute_class_priors and compute_feature_likelihoods. The log probabilities will be stored in the 'predictions' array.

In [11]:
from scipy.special import logsumexp


def NB_classify(class_priors, likelihoods, X_val):

    predictions = np.zeros((X_val.shape[0], num_classes))  # an empty numpy array to store the predictions in

    sum_of_log_likelihoods = X_val.dot(np.log(likelihoods))
    log_joint_prob = sum_of_log_likelihoods + np.log(class_priors)[None, :]
    for n, doc in enumerate(X_val):
        predictions[n, :] = log_joint_prob[n]
        predictions[n, :] -= logsumexp(predictions[n, :])
    return predictions

predictions = NB_classify(class_priors, likelihoods, X_val)
print(predictions)

[[-1.67995956e+00 -2.06268362e-01 -1.24848524e+01]
 [-2.44930902e-08 -1.75248754e+01 -3.48347877e+01]
 [-2.71925657e+01 -5.93553923e-03 -5.12976370e+00]
 [-2.13926690e+01 -1.29194694e+01 -2.45040019e-06]
 [-1.71896156e+01 -4.03621812e-08 -1.89127774e+01]
 [-1.35507627e+01 -5.09757661e+00 -6.13160229e-03]
 [-2.64843584e+01 -5.17194056e+00 -5.68970399e-03]
 [-1.71412353e-07 -1.55791940e+01 -4.34976594e+01]
 [-6.07452216e+00 -8.49093621e+00 -2.50921170e-03]
 [-1.17698235e-03 -6.74538989e+00 -2.54962863e+01]
 [-5.90396512e+00 -2.33897328e-02 -3.89272496e+00]
 [-7.78788353e-06 -1.17629453e+01 -4.26526674e+01]
 [-4.99002650e+00 -6.83215701e-03 -1.25956374e+01]
 [-2.60255234e-06 -1.33436449e+01 -1.38159437e+01]
 [-1.51863737e-03 -6.49070104e+00 -3.22236283e+01]
 [-4.90956654e+00 -1.02770961e-02 -5.86086358e+00]
 [-1.76328837e+01 -8.79473987e-08 -1.65341879e+01]
 [-3.41376905e+01  0.00000000e+00 -3.64717215e+01]
 [-5.63711560e-01 -1.05828660e+00 -2.47864057e+00]
 [-1.06097103e-05 -1.15519618e+

Use the 'predictions' array above to compute and print the accuracy of the classifier on the validation set.   

In [12]:
from sklearn.metrics import accuracy_score

accuracy_score(val_labels, np.argmax(predictions, axis=1))

0.715

## TASK 1.1c

The simplicty of naïve Bayes means that we can quite easily interpret the model. In the code above, we used the functions you implemented, compute_feature_likelihoods and compute_class_priors, to train an NB classifier with our training set. Given this classifier, which are the five n-gram features that most strongly indicate that the document belongs to class 0? Store these features in the 'top_features' list below.    (4 marks)

In [13]:
top_features = []

### WRITE YOUR ANSWER HERE

# Get the log likelihoods
log_likelihoods = np.log(likelihoods)

# Calculate log odds: log(P(feature | class 0)) - log(P(feature | class 1))
log_odds = log_likelihoods[:, 0] - log_likelihoods[:, 1]

# Get the indices of the top 5 features with the highest log odds
top_5_indices = np.argsort(log_odds)[-5:][::-1]

# Get the n-gram features corresponding to these indices
top_features = [vectorizer.get_feature_names_out()[i] for i in top_5_indices]
### END OF ANSWER

### DO NOT MODIFY
print(top_features)
pd.DataFrame(top_features).to_csv('./outputs/11c_top_feats.csv')
###

['physical risks', 'weather events', 'extreme weather', 'the physical', 'change is']


Up to this point, the classifier used bigrams features extracted using CountVectorizer with the wordnet tokenizer. 

## TASK 1.1d

Your task is to improve the naïve Bayes classifier by changing the preprocessing or features only. It is up to you to decide how many changes are needed to improve the classifier -- a single change may be enough to achieve a good result (and maximum marks) and you should only include steps that help performance. Complete the 'preprocess_improved' function below, and run the cell to compute accuracy of the improved classifier on the validation set.     (3 marks)

In [14]:
def preprocess_improved(train_texts):
    ### WRITE YOUR ANSWER HERE

    vectorizer = CountVectorizer(ngram_range=(1,2), tokenizer=word_tokenize)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    ### END OF ANSWER

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess_improved(train_texts)
class_priors = compute_class_priors(train_texts, train_labels)
likelihoods = compute_feature_likelihoods(X, train_labels)
predictions = NB_classify(class_priors, likelihoods, X_val)
predictions_nb = np.argmax(predictions, axis=1)

### DO NOT MODIFY
pd.DataFrame(predictions_nb).to_csv('./outputs/11d_improved_preds.csv')
accuracy_improved = accuracy_score(val_labels, predictions_nb)
print(accuracy_improved)
###

C:\Users\agarw\anaconda3\envs\text_analytics\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.77


## TASK 1.2

Below is an implementation of a neural network classifier that we can apply to the same dataset. However, there are some mistakes in the code and some poor choices in the choice of parameters and architecture. Your task is to fix the errors, make better parameter choices, and improve the model's performance. **Modify the code within the next cell** to improve the neural network classifier, then run it and compute its accuracy using the code in the cell after that.   (8 marks)

In [15]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE: MODIFY THE CODE WITHIN THIS CELL
 
from torch import nn
from torch import optim
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")  

sequence_length = 128  # truncate all docs longer than this. Pad all docs shorter than this.
batch_size = 32

def tokenize_function(examples):
    return tokenizer(
        examples["text"],  # Adjust the key based on your dataset structure
        padding="max_length",  # Ensures equal sequence lengths
        truncation=True,       # Truncates longer sequences
        max_length=sequence_length,        # Adjust as needed
        return_tensors="pt"
    )

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])  # Adjust column names
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)

tokenized_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])  # Adjust column names
val_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False)

class FFTextClassifier(nn.Module):
    
    def __init__(self, vocab_size, sequence_length, num_classes, embedding_size=100, hidden_size=128):
        super(FFTextClassifier, self).__init__()

        self.embedding_size = embedding_size
        self.sequence_length = sequence_length

        # Here we just need to construct the components of our network. We don't need to connect them together yet.
        self.embedding_layer = nn.Embedding(vocab_size, embedding_size) # embedding layer
        # self.hidden_layer = nn.Linear(self.embedding_size,sequence_length, hidden_size)  #nn.LSTM(self.embedding_size, hidden_size, bidirectional=True, batch_first=True) # Hidden layer
        self.hidden_layer = nn.Linear(self.embedding_size, hidden_size)  #nn.LSTM(self.embedding_size, hidden_size, bidirectional=True, batch_first=True) # Hidden layer
        self.dropout = nn.Dropout(0.3)
        
        self.activation = nn.ReLU() # Hidden layer
        self.output_layer = nn.Linear(hidden_size, num_classes) # Full connection layer
        
        
    def forward(self, input_words):
        # Input dimensions are:  (batch_size, seq_length)
        embedded_words = self.embedding_layer(input_words)  # (batch_size, seq_length, embedding_size)
       
        pooled = embedded_words.mean(dim=1)  # (batch, emb_size)
       
        z = self.hidden_layer(pooled)

        # flatten the sequence of embedding vectors for each document into a single vector.
        # embedded_words = embedded_words.reshape(embedded_words.shape[0], self.sequence_length*self.embedding_size)  #(embedded_words.shape[0], self.sequence_length*self.embedding_size)  # batch_size, seq_length*embedding_size

        # z = self.hidden_layer(embedded_words)   # (batch_size, seq_length, hidden_size)
        
        h = self.activation(z)                 # (batch_size, seq_length, hidden_size)
        h = self.dropout(h)
        output = self.output_layer(h)                      # (batch_size, num_classes)

        # Notice we haven't applied a softmax activation to the output layer -- it's not required by Pytorch's loss function.

        return output


    def run_training(self, train_dataloader, dev_dataloader):

        # training hyperparameters
        num_epochs = 49
        learning_rate =3e-4  # learning rate for the gradient descent optimizer, related to the step size

        loss_fn = nn.CrossEntropyLoss()  # create loss function object
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)  # create the optimizer
        
        dev_losses = []
            
        for e in range(num_epochs):
            # Track performance on the training set as we are learning...
            train_losses = []

            self.train()  # Put the model in training mode.

            for i, batch in enumerate(train_dataloader):
                # Iterate over each batch of data

                optimizer.zero_grad()  # Reset the optimizer

                # Use the model to perform forward inference on the input data.
                # This will run the forward() function.
                output = self(batch['input_ids'])

                # Compute the loss for the current batch of data
                batch_loss = loss_fn(output, batch['label'])

                # Perform back propagation to compute the gradients with respect to each weight
                batch_loss.backward()

                # Update the weights using the compute gradients
                optimizer.step()

                # Record the loss from this sample to keep track of progress.
                train_losses.append(batch_loss.item())

            print("Epoch: {}/{}".format((e+1), num_epochs),
                "Training Loss: {:.4f}".format(np.mean(train_losses)))

            self.eval()  # Switch model to evaluation mode

            dev_losses_epoch = []
            
            for dev_batch in dev_dataloader:
                dev_output = self(dev_batch['input_ids'])
                dev_loss = loss_fn(dev_output, dev_batch['label'])

                # Save the loss on the dev set
                dev_losses_epoch.append(dev_loss.item())
                        
            dev_losses.append(np.mean(dev_losses_epoch))
                    
            print("Epoch: {}/{}".format((e+1), num_epochs),
                "Validation Loss: {:.4f}".format(dev_losses[-1]) )

        return dev_losses

def predict_nn(trained_model, data_loader):

    trained_model.eval()

    pred_labs = []  # predicted labels to return
    
    for batch in data_loader:
        test_output = trained_model(batch['input_ids'])
        predicted_labels = test_output.argmax(1)
        pred_labs.extend(predicted_labels.tolist())
    
    return pred_labs

vocab_size = len(tokenizer.get_vocab())

# vocab_size = len(tokenizer.get_vocab().values()) + 1
nn_classifier_model = FFTextClassifier(vocab_size, sequence_length, num_classes)
dev_losses = nn_classifier_model.run_training(train_loader, val_loader)

predictions_nn = predict_nn(nn_classifier_model, val_loader)

### END OF ANSWER 

### DO NOT MODIFY
pd.DataFrame(predictions_nn).to_csv("./outputs/12_nn_preds.csv")
accuracy_nn = accuracy_score(val_labels, predictions_nn)
print(accuracy_nn)
###

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch: 1/49 Training Loss: 1.0834
Epoch: 1/49 Validation Loss: 1.0577
Epoch: 2/49 Training Loss: 1.0707
Epoch: 2/49 Validation Loss: 1.0463
Epoch: 3/49 Training Loss: 1.0656
Epoch: 3/49 Validation Loss: 1.0430
Epoch: 4/49 Training Loss: 1.0538
Epoch: 4/49 Validation Loss: 1.0391
Epoch: 5/49 Training Loss: 1.0531
Epoch: 5/49 Validation Loss: 1.0329
Epoch: 6/49 Training Loss: 1.0473
Epoch: 6/49 Validation Loss: 1.0258
Epoch: 7/49 Training Loss: 1.0410
Epoch: 7/49 Validation Loss: 1.0217
Epoch: 8/49 Training Loss: 1.0332
Epoch: 8/49 Validation Loss: 1.0125
Epoch: 9/49 Training Loss: 1.0212
Epoch: 9/49 Validation Loss: 0.9994
Epoch: 10/49 Training Loss: 1.0094
Epoch: 10/49 Validation Loss: 0.9920
Epoch: 11/49 Training Loss: 0.9960
Epoch: 11/49 Validation Loss: 0.9781
Epoch: 12/49 Training Loss: 0.9804
Epoch: 12/49 Validation Loss: 0.9622
Epoch: 13/49 Training Loss: 0.9546
Epoch: 13/49 Validation Loss: 0.9478
Epoch: 14/49 Training Loss: 0.9309
Epoch: 14/49 Validation Loss: 0.9248
Epoch: 15/

We now explore the use of transformers for building a text classifier. First, let's look at how the process a document. We'll chose one at random from the training set:

In [16]:
### DO NOT MODIFY
chosen_document = train_texts[np.random.randint(len(train_texts))]

## TASK 1.3a

Use the HuggingFace transformers library to load the pretrained BERT model "prajjwal1/bert-tiny". Obtain a document embedding for the chosen document given above. Comment your code to explain how it obtains a representation of the document.    (3 marks)

In [17]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE

from transformers import BertTokenizer, BertModel

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
model = BertModel.from_pretrained('prajjwal1/bert-tiny')


doc_emb = []  # use this variable to store the document embedding


# Tokenize the chosen document
inputs = tokenizer(chosen_document, return_tensors='pt', truncation=True, padding=True, max_length=512)

# Obtain the model output (embedding)
with torch.no_grad():  # No need to compute gradients for embedding extraction
    outputs = model(**inputs)

# Extract the last hidden state (document representation)
# outputs.last_hidden_state contains the token embeddings, we average them to get a single document embedding
doc_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

# Store the document embedding
doc_emb = doc_embedding

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(doc_emb).to_csv('./outputs/13a_sen_emb.csv')
print(doc_emb)
###

[-0.03521617501974106, 0.5762357115745544, -0.8187068700790405, -0.7629048824310303, 0.3920254707336426, 0.04580053314566612, 0.10891719907522202, -0.02957385778427124, -1.6152565479278564, -0.29160165786743164, 0.11950462311506271, 0.6467519402503967, -0.3796098530292511, -0.806648313999176, 1.4413185119628906, -0.14654335379600525, -0.9145377278327942, -0.3485006093978882, -0.5447779893875122, 0.4868704080581665, 0.335915207862854, -0.00711021525785327, 0.6897221803665161, 1.340092420578003, 1.5206325054168701, -0.6822413802146912, 0.32936936616897583, 0.33143436908721924, -0.3640090227127075, -0.5021382570266724, -0.8672389984130859, -2.0557408332824707, -0.1609272062778473, -0.11616913974285126, 0.9012329578399658, -1.289573073387146, -0.72501140832901, 0.03544306755065918, -2.4419004917144775, -0.3286975026130676, 0.6415501832962036, 0.18586714565753937, -0.06796848028898239, -1.132178783416748, -0.2014036476612091, -1.4121601581573486, 0.06083305925130844, 0.7349420785903931, -0.

## TASK 1.3b

Using the same document embeddings method as the previous task (1.3a), find the most similar document to the 'chosen_document' from within the validation set (from the 'val_texts' object). Use a standard similarity metric that considers the direction but not the magnitude of the embedding vectors.  (2 marks)

In [18]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from sklearn.metrics.pairwise import cosine_similarity

most_similar_doc = ""  # use this variable to store the most similar document

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
model = BertModel.from_pretrained('prajjwal1/bert-tiny')

# Function to get document embedding
def get_doc_embedding(document):
    # Tokenize the document
    inputs = tokenizer(document, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Obtain model output (embedding)
    with torch.no_grad():
        outputs = model(**inputs)
    # Take the mean of the token embeddings to get a document embedding
    doc_embedding = outputs.last_hidden_state[0,0,:].numpy()
    return doc_embedding

# Get the embedding for the chosen document
chosen_doc_embedding = get_doc_embedding(chosen_document)

# Get embeddings for all documents in the validation set
val_embeddings = [get_doc_embedding(doc) for doc in val_texts]

# Calculate cosine similarity between the chosen document and each document in the validation set
similarities = cosine_similarity([chosen_doc_embedding], val_embeddings)[0]

# Find the index of the most similar document
most_similar_idx = np.argmax(similarities)

# Get the most similar document from the validation set
most_similar_doc = val_texts[most_similar_idx]
### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame([chosen_document, most_similar_doc]).to_csv("./outputs/13b_most_similar.csv")
print(chosen_document)
print(most_similar_doc)
###

In 2019, the Group elected to discontinue financing power generation companies in Poland, given that their power mix is highly dependent on coal and the Bank found, after a two-year commitment, that they had no intention of changing their strategy.
In terms of thermal coal, the Group has set an exit deadline, in line with the SDS (Sustainable Develop- ment Scenario) scenario of the International Ener- gy Agency (IEA), compatible with the climate goals of the Paris Agreement: In 2019 and in 2020, the Group strengthened its position on coal, announ- cing its plan to reduce its thermal coal exposure to zero by 2030 in OECD countries, and by 2040 in the rest of the world.


## TASK 1.3c

Implement a classifier based on the same pretrained transformer model, "prajjwal1/bert-tiny". Evaluate your model's performance on the validation set. Use an 'auto class' from HuggingFace to build your classifier (see https://huggingface.co/docs/transformers/model_doc/auto).   (6 marks)

In [ ]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

predictions_bert = []  # use this variable to store the predicted labels for the validation set

# Load tokenizer and model for sequence classification
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-tiny')
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=num_classes)  # Assuming binary classification

# Tokenize the dataset
def tokenize_function(text):
    return tokenizer(text["text"], padding="max_length", truncation=True,max_length=512, return_tensors="pt")

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./outputs",          # Output directory
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=16,   # Batch size for training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    num_train_epochs=17,              # Number of training epochs
    weight_decay=0.01,               # Weight decay for regularization
)

# Define the Trainer
trainer = Trainer(
    model=model,                          # The pretrained model
    args=training_args,                   # Training arguments
    train_dataset=train_dataset,          # Training dataset
    eval_dataset=val_dataset,             # Validation dataset
    compute_metrics=lambda p: {"accuracy": accuracy_score(p.predictions.argmax(axis=-1), p.label_ids)},  # Evaluation metric
)

# Train the model
trainer.train()

# Make predictions on the validation set
predictions = trainer.predict(val_dataset)


# Convert predictions to labels
predicted_labels = predictions.predictions.argmax(axis=-1).tolist()

# Store the predictions
predictions_bert = predicted_labels
### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_bert).to_csv('./outputs/13c_bert_preds.csv')
accuracy_tinybert = accuracy_score(val_dataset["label"], predictions_bert)
print(accuracy_tinybert)
###

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

C:\Users\agarw\anaconda3\envs\text_analytics\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.993925,0.535000
2,No log,0.883211,0.710000
3,No log,0.780866,0.725000
4,No log,0.691867,0.770000
5,No log,0.641258,0.790000
6,No log,0.620770,0.735000
7,No log,0.592918,0.785000
8,No log,0.572927,0.780000
9,No log,0.554442,0.760000
10,0.666600,0.543738,0.760000
